# This notebook contains a walkthrough of how to load in the game prediction model, and then download today's games and the corresponding spreads, and does model.predict or something to provide a suggestion with x% confidence. 

In [28]:
#import dependencies
from create_dataset import create_rolling_dataset  
from keras.models import Sequential
from keras.layers import Dense


Using TensorFlow backend.


In [29]:
#rolling window size   
WINDOW = 5

X , y = create_rolling_dataset(window=WINDOW,FILENAME='NBADATA.csv',winner=True)

/Users/noahkasmanoff/NBAForecasting/2k19_Forecast/create_dataset.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  team_season['Rolling ' + col] = team_season[col].rolling(window=N_GAMES).mean().shift(1)
/Users/noahkasmanoff/NBAForecasting/2k19_Forecast/create_dataset.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  road_df['road_' + col] = road_df[col]
/Users/noahkasmanoff/NBAForecasting/2k19_Forecast/create_dataset.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 12)
scaler  = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [31]:
from sklearn.neural_network import MLPClassifier


model = MLPClassifier()
model.fit(X_train,y_train)

print("Test: ", model.score(X_train,y_train))
print("score: ", model.score(X_test,y_test))

Test:  0.684830293235
score:  0.679362880886


In [32]:
from sklearn.svm import SVC


model = SVC()
model.fit(X_train,y_train)

print("Test: ", model.score(X_train,y_train))
print("score: ", model.score(X_test,y_test))

Test:  0.68898637728
score:  0.691828254848


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 12)
scaler  = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#split and normalize data. 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(30, input_dim=X_train.shape[1], activation='relu'))
#note softmax is used for classification problems as the final entry before the output. 

model.add(Dense(30, activation='sigmoid'))

model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#need to use a large number of epochs for this set, still end up with non overfit data based on the validation reisutls plus model evaluation. 
model.fit(X_train,y_train,batch_size=10,epochs=20,validation_split=0.2,verbose=1)



In [ ]:
model.evaluate(X_test,y_test)[1]

In [1]:
import scrape_today

In [2]:
matchups = scrape_today.get_todays_games(16,10,2018)

In [3]:
matchups

[['PHI', 'BOS'], ['OKC', 'GSW']]

In [4]:
scrape_today.predict_todays_games()

NameError: name 'prediction_stuff' is not defined

In [6]:
matchups[0]

['PHI', 'BOS']

In [23]:
def get_splits(matchup,ten=False,twenty=False,regseason=True):
    road_team = matchup[0]
    home_team = matchup[1]
    import numpy as np
    import pandas as pd
    from nba_py import team
    teams = team.TeamList()
    teamids = teams.info()
    teamids = teamids[:-15]
    teamids = teamids[['TEAM_ID','ABBREVIATION']]

    teamids = teamids.rename(index=str, columns={"ABBREVIATION": "Team"})
    teamids = teamids.replace('BKN','BRK')
    teamids = teamids.sort_values('Team')

    TEAM_ID = teamids.loc[teamids['Team'] == road_team].values[0,0]
    teamids = pd.get_dummies(teamids)
    teamarray = teamids.loc[teamids['TEAM_ID'] == TEAM_ID].values[0,1:]


    TEAM = team.TeamLastNGamesSplits(team_id=TEAM_ID,season='2017-18')
    if ten:
        df = TEAM.last10()
    if twenty:
        df = TEAM.last20()
    if regseason:
        TEAM = team.TeamGeneralSplits(team_id=TEAM_ID,season='2017-18')
        df = TEAM.overall()
   # if five:
        #df = TEAM
    
    df = df[['OREB','DREB','REB','PF','STL','TOV','BLK','FG3_PCT','FG_PCT','FT_PCT']].values
    
    road_teamsplits = df#.values   #np.concatenate((df[0],teamarray))
    
    teams = team.TeamList()
    teamids = teams.info()
    teamids = teamids[:-15]
    teamids = teamids[['TEAM_ID','ABBREVIATION']]

    teamids = teamids.rename(index=str, columns={"ABBREVIATION": "Team"})
    teamids = teamids.replace('BKN','BRK')
    teamids = teamids.sort_values('Team')

    
    TEAM_ID = teamids.loc[teamids['Team'] == home_team].values[0,0]
    teamids = pd.get_dummies(teamids)
    teamarray = teamids.loc[teamids['TEAM_ID'] == TEAM_ID].values[0,1:]


    TEAM = team.TeamLastNGamesSplits(team_id=TEAM_ID,season='2017-18')
    if ten:
        df = TEAM.last10()
    if twenty:
        df = TEAM.last20()
    if regseason:
        TEAM = team.TeamGeneralSplits(team_id=TEAM_ID,season='2017-18')
        df = TEAM.overall()
   # if five:
        #df = TEAM
    
    df = df[['OREB','DREB','REB','PF','STL','TOV','BLK','FG3_PCT','FG_PCT','FT_PCT']].values
    home_teamsplits = df#.values  # np.concatenate((df[0],teamarray))
    teamsplits = np.concatenate((road_teamsplits,home_teamsplits))
    return teamsplits

In [24]:
six_celts = get_splits(matchups[0],ten=True)

In [25]:
six_celts.shape

(2, 10)

In [26]:
six_celts

array([[ 10.9  ,  36.5  ,  47.4  ,  22.1  ,   8.3  ,  16.5  ,   5.1  ,
          0.369,   0.472,   0.752],
       [  9.4  ,  35.1  ,  44.5  ,  19.7  ,   7.4  ,  14.   ,   4.5  ,
          0.377,   0.45 ,   0.771]])